In [1]:
import os
os.environ['PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT'] = '2'

In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Set the option to prevent the FutureWarning
pd.set_option('future.no_silent_downcasting', True)
import datetime as dt
from functions import *
from dtype_dictionaries import *

### Constants

In [3]:
# Position of the gtfs post-rating schedule files
gtfs_post_rating_files = os.listdir('gtfsSchedule')
# Sort the elements in alphabetical order. I need to ensure this otherwise the strategy of keeping the latest df in memory won't work
gtfs_post_rating_files.sort()

# List only the files needed later, avoid to import the whole folder
txt_list = (['calendar.txt', 'calendar_attributes.txt', 'calendar_dates.txt', 'feed_info.txt', 'routes.txt', 'stop_times.txt', 'stops.txt', 'trips.txt'])
# Position of MBTA_ArrivalDepartureTimes files
ArrDepFolder = 'MBTA_ArrivalDepartureTimes'
# Position of the  parsed ArrivalDepartureTimes files
parsed_ArrDepFolder = 'parsed_ArrivalDepartureFiles'
# List of files contained in the parsed_ArrivalDepartureFiles folder
parsed_gtfs_rt = os.listdir(parsed_ArrDepFolder)
parsed_gtfs_rt.sort()

chunk_size = 12**5

# Initialize to store calendar days and related services, to be converted to dataframe
calendar_service_map = []
# Initialize the list of dataframes to be concatenated containing all the data from gtfs rt
df_ArrDep_list = []
# Initialize the list of gtfs_schedules to be concatenated
gtfs_schedules_list = []

start_date_list = []
end_date_list = []
# Read files or import table?
import_separate_files = False

# Create two lists to store the unmatched names and groups
unmatched_names = []
unmatched_groups = []

# Variable we wish to groupby
grouping_vars = ['direction_id', 'scheduled']

### Import files with arrival and departure times

In [5]:
for postRatingRecap_file in gtfs_post_rating_files:
    adt_list = []
    print(f'Processing {postRatingRecap_file}...')
    gtfs_post_rating_folder = os.path.join('gtfsSchedule', postRatingRecap_file)
    calendar, calendar_attributes, calendar_dates, feed_info, routes, stop_times, stops, trips, gtfs_schedule = get_gtfs_post_rating_txt_files(gtfs_post_rating_folder, txt_list, gtfs_cols)
    # Add schedule to the list of gtfs_schedules
    gtfs_schedules_list.append(gtfs_schedule)    
    # Save feed_start_date and feed_end_date
    start_date = feed_info['feed_start_date'][0]
    start_date_list.append(start_date)
    end_date = feed_info['feed_end_date'][0]
    end_date_list.append(end_date)
    calendar_data = parse_calendar_file(calendar)
    calendar_data = parse_calendar_dates_file(calendar_dates, calendar_data)
    calendar_schedule = generate_schedule(start_date, end_date, calendar_data)
    calendar_service_map.append(calendar_schedule)

    if import_separate_files:

        # Return the list of compatible files
        compatibleFiles = get_compatible_files(ArrDepFolder, start_date, end_date)
        for filename in (compatibleFiles):   
            print(f'Importing {filename}...')  
            for chunk in pd.read_csv(filename, chunksize=chunk_size, dtype=adt_dtype_map, low_memory=False):
                # Carry out here any filtering, drop or cutting down operation
                chunk = reduce_df_size(chunk)
                chunk = adjust_adt_df_settings(chunk, routes, start_date, end_date)
                adt_list.append(chunk)

        print('Concatenating...')
        #adt_df = pd.concat(adt_list, axis=0)
        # Filter out all the records that lie outside the feed_start_date and feed_end_date rang
        #adt_df = adjust_adt_df_settings(adt_df, routes, start_date, end_date)
        # Save the dataframe to a csv file: filename is equal as 'feed_start_date_feed_end_date.csv'
        export_filename = start_date.strftime('%Y%m%d') + '_' + end_date.strftime('%Y%m%d') + '.csv'
        filepath = os.path.join('parsed_ArrivalDepartureFiles', export_filename)
        print(f'Exporting {filepath}...')
        pd.concat(adt_list, axis=0).to_csv(filepath, index=False)

    else:        
        # Get list index of postRatingRecap_file
        idx = gtfs_post_rating_files.index(postRatingRecap_file)
        parsed_file = parsed_gtfs_rt[idx]
        print(f'Importing {parsed_file}...') 
        filepath = os.path.join(parsed_ArrDepFolder, parsed_file)
        for chunk in pd.read_csv(filepath, chunksize=chunk_size, dtype=adt_dtype_map, low_memory=False):
            chunk['service_date'] = pd.to_datetime(chunk['service_date'], format='%Y-%m-%d')
            chunk['scheduled'] = pd.to_datetime(chunk['scheduled'], format='%H:%M:%S')
            chunk['actual'] = pd.to_datetime(chunk['actual'], format='%H:%M:%S')            
            adt_list.append(chunk)

        # Concatenate the list of dataframes
        adt_df = pd.concat(adt_list, axis=0, ignore_index=True)
        # Map realtime data to gtfs schedule to assign block_ids and service_ids
        adt_df = map_realtime_to_gtfs_schedule(adt_df, start_date, end_date, calendar_schedule, gtfs_schedule)
print('Concatenating...')      
adt_df = pd.concat(adt_list, axis=0, ignore_index=True)
# Build the calendar_service_map and gtfs_schedule dataframe
# Convert every element in calendar_service_map to a dataframe
calendar_df = pd.concat([pd.DataFrame(map, columns=['date', 'day_of_week', 'service_ids']) for map in calendar_service_map], axis=0, ignore_index=True)

Processing 01_gtfs_post-recap_Winter_2022...
Importing 20211219_20220312.csv...
Processing route 1...
Feed start date: 2021-12-19 00:00:00, Feed end date: 2022-03-12 00:00:00


c:\Users\antonio.forte\Dropbox (MIT)\GitHub\getGTFS-RT\MBTA_PostRatingRecap_ArrDepTimes\functions.py:288: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['WinterSunday', 'ChristmasDay-NewYearsDay', 'WinterSunday', 'ChristmasDay-NewYearsDay', 'WinterSunday', ..., 'WinterSunday', 'WinterSunday', 'WinterSunday', 'WinterSunday', 'WinterSunday']
Length: 14
Categories (129, object): ['BUS122-hba12ns1-Wdy-02', 'BUS122-hbb12ns1-Wdy-02', 'BUS122-hbc12ns1-Wdy-02', 'BUS122-hbf12ns1-Wdy-02', ..., 'RTL122-hms12fn1-Wdy-01', 'WinterSaturday', 'WinterSunday', 'WinterWeekday']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[merged.index, 'service_id'] = merged['service_id_y']
c:\Users\antonio.forte\Dropbox (MIT)\GitHub\getGTFS-RT\MBTA_PostRatingRecap_ArrDepTimes\functions.py:289: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in

KeyboardInterrupt: 

### Combine arrival and departure times with scheduled information

In [ ]:
adt_df.block_id = ''
adt_df.service_id = ''
adt_df_copy = adt_df.copy()

route_ids = adt_df_copy['route_id'].unique()
# From the start and end date lists, extract the feed_start_date and feed_end_date and convert them to datetime ranges
feed_start_date = pd.to_datetime(start_date_list, format='%Y%m%d')
feed_end_date = pd.to_datetime(end_date_list, format='%Y%m%d')

for route in route_ids[0:1]:
    print(f'Processing route {route}...')
    # Fetch subset of the ArrivalDepartureTimes dataframe for the current route and stop_sequence ==1
    adt_route = adt_df_copy.loc[(adt_df_copy['route_id'] == route) & (adt_df_copy['point_type'] == 'Startpoint')]

    for feeds in zip(feed_start_date[0:1], feed_end_date[0:1], calendar_service_map[0:1], gtfs_schedules_list[0:1]):
        # Print the feed_start_date and feed_end_date
        print(f'Feed start date: {feeds[0]}, Feed end date: {feeds[1]}')
        #Filter adt_df to keep only the rows that lie within the feed_start_date and feed_end_date range
        date_filter = (adt_route['service_date'] >= feeds[0]) & (adt_route['service_date'] <= feeds[1])
        adt_grouped = adt_route[date_filter]
        service_map = feeds[2]
        gtfs_schedule = feeds[3]
        # Fetch subset of the GTFS schedule dataframe for the current route
        schedule_route = gtfs_schedule[(gtfs_schedule['route_id'] == route)&(gtfs_schedule['stop_sequence'] == 1)]

        adt_grouped = adt_grouped.groupby(grouping_vars, observed=True)
        schedule_grouped = schedule_route.groupby(grouping_vars, observed=True)
    
        for name, group in adt_grouped:
            # print the group name
            print(f'{name}...')
            if name in schedule_grouped.groups:
                # extract the corresponding group from schedule_route10_grouped
                schedule_group = schedule_grouped.get_group(name)
                schedule_services = set(schedule_group['service_id'])
                # This is a series whose index is the service_id and the values are the block_ids
                schedule_service_block_ids = schedule_group.groupby(['service_id'], observed=True, as_index=False)['block_id'].apply(list)

                # extract the subset of the calendar_df that matches the service_date
                service_days_orig = calendar_df.loc[calendar_df.date.isin(group.service_date)]

                # loop through the service_days
                # Step 1: Compute the intersections
                service_days = service_days_orig.copy()
                service_days.loc[:,'adt_service_ids'] = service_days['service_ids'].apply(lambda ids: schedule_services.intersection(ids))
                service_days.loc[:,'adt_service_ids_str'] = service_days['adt_service_ids'].apply(lambda ids: ', '.join(ids))

                # Step 2: Merge service_days with group on date 
                merged = pd.merge(group, service_days, left_on='service_date', right_on='date')
                # Step 3: Merge the merged dataframe with schedule_service_block_ids and keep the index of the group
                merged = pd.merge(merged, schedule_service_block_ids, left_on='adt_service_ids_str', right_on='service_id')
                # Print the name of the group if the length of the two indexes is different to spot potential errors
                if len(merged.index) != len(group.index):
                    print(f'Route: {route}\n Length of indexes for group {name} is different: {len(merged.index)} vs {len(group.index)}')
                    # Drop group rows whose service_date is not in the service_date of merged
                    group = group[group['service_date'].isin(merged['service_date'])]

                merged = merged.set_index(group.index)
                # Step 4: recast the service_id and block_id in the adt_df
                adt_df_copy.loc[merged.index, 'service_id'] = merged['service_id_y'] 
                adt_df_copy.loc[merged.index, 'block_id'] = merged['block_id_y']

            else:
                #print(f'No match found for group {name}...')
                unmatched_names.append(name)
                unmatched_groups.append(group)

    # Save the route-specific df to a csv file
    export_filename = route + '.csv'
    filepath = os.path.join('block_ids_df', export_filename)
    # Keep the index in the exported csv file and its name half_trip_id
    #adt_df_copy.loc[(adt_df_copy.route_id == route) & (adt_df_copy['point_type'] == 'Startpoint')].to_csv(filepath, index=True)
    endpoint_df = adt_df_copy.loc[adt_df_copy['point_type'] == 'Endpoint']
    startpoint_df = adt_df_copy.loc[adt_df_copy['point_type'] == 'Startpoint', ['half_trip_id','block_id', 'service_id']]
    endpoint_df.merge(startpoint_df, how='left', left_index=True, right_index=True, suffixes=('_end', '_start'))
# I want to extract the subset containing only Endpoint rows that do not have any block_id nor service_id
# I will subsequently merge this df with that containing only Startpoint that was just processed (it takes much less time)
# And finally, I will concatenate the two df horizontally and obtain the final df


In [ ]:
adt_route1_copy = adt_route1.copy()
endpoint_df = adt_route1_copy.loc[adt_route1_copy['point_type'] == 'Endpoint']
# Drop block_id and service_id columns
endpoint_df = endpoint_df.drop(columns=['block_id', 'service_id'])
startpoint_df = adt_route1_copy.loc[adt_route1_copy['point_type'] == 'Startpoint', ['half_trip_id','block_id', 'service_id']]
startpoint_df
endpoint_df = endpoint_df.merge(startpoint_df, left_on='half_trip_id', right_on='half_trip_id')
# Concatenate the two dataframes horizontally
final_df = pd.concat([startpoint_df,endpoint_df], axis=1)
final_df

In [ ]:
for i, row in service_days.iterrows():
                    # add the intersection between schedule_services and row['service_ids'] to the service_id column in adt_route10 as plain strings
                    adt_service_ids = schedule_services.intersection(row['service_ids'])
                    adt_service_ids_str = ', '.join(adt_service_ids)  # Convert set to string
                    calendar_idx = group.loc[group.service_date == row.date].index
                    half_trip_id_value = adt_df.loc[calendar_idx, 'half_trip_id'].values[0]
                    # Assign service_id to row with Startpoint
                    adt_df.loc[calendar_idx, 'service_id'] = adt_service_ids_str
                    # Assign half_trip_id to row with Endpoint --> this is the same as the half_trip_id of the corresponding Startpoint
                    adt_df.loc[adt_df.half_trip_id == half_trip_id_value, 'service_id'] = adt_service_ids_str
                    # Get the block_id list associated to adt_service_ids_str
                    block_list=schedule_service_block_ids[schedule_service_block_ids.index==adt_service_ids_str]
                    block_list = block_list.iloc[0] if not block_list.empty else ''

                    if block_list and not group.empty:
                        # Concatenate block_list elements into a comma-separated string
                        block_ids_str = ', '.join(block_list)
                        # Assign the concatenated string to the Startpoint row
                        adt_df.loc[calendar_idx, 'block_id'] = block_ids_str
                        # Assign the concatenated string to the Endpoint row
                        adt_df.loc[adt_df.half_trip_id == half_trip_id_value, 'block_id'] = block_ids_str

In [ ]:
adt_df_copy.loc[adt_df_copy.route_id=='1']

In [ ]:
print('Split Blocks')
adt_df_block_ids = split_multiple_block_id(adt_df_block_ids)
print('Assign Blocks to endpoints')

In [ ]:
# Assign the same values of block_id and service_id to Endpoint records with the same half_trip_id
trips_grouped = adt_df.groupby(['half_trip_id'], observed=True)
for name, group in trips_grouped:
    if len(group.index) > 1:
        adt_df.loc[group.index[1], ['block_id', 'service_id']] = group.loc[group.index[0], ['block_id', 'service_id']].values
    else:
        print(name)
        print(group)

In [ ]:
trips_grouped = adt_df.groupby(['half_trip_id'], observed=True)
for name, group in trips_grouped:
    if len(group.index) > 1:
        adt_df.loc[group.index[1], ['block_id', 'service_id']] = group.loc[group.index[0], ['block_id', 'service_id']].values
    else:
        print(name)
        print(group)

In [ ]:
# Assign the same values of block_id and service_id to Endpoint records with the same half_trip_id
trips_grouped = adt_df.groupby(['half_trip_id'], observed=True)
for name, group in trips_grouped:
    adt_df.loc[group.index[1], ['block_id', 'service_id']] = group.loc[group.index[0], ['block_id', 'service_id']].values

### Compute layover

In [ ]:
# Create layover_df as a copy
layover_df = adt_route10.copy()
layover_df = layover_df.sort_values(by=['block_id','service_date','half_trip_id','departure_time'])
layover_df=layover_df.reset_index(drop=True)

# Group by 'block_id' and 'service_date', skipping rows with null 'service_id' or 'block_id'
grouped = layover_df.loc[layover_df.block_id.notna()].groupby(['block_id', 'service_date'])

# Calculate theoretical and actual layover times using diff()
layover_df['theoretical_layover'] = grouped['departure_time'].diff().dt.total_seconds() / 60
layover_df['actual_layover'] = grouped['actual'].diff().dt.total_seconds() / 60

# Replace the first row of each group with null timedelta
layover_df.loc[grouped.head(1).index, ['theoretical_layover', 'actual_layover']] = 0
layover_df.loc[layover_df.time_point_order != 1, ['theoretical_layover', 'actual_layover']] = np.nan
layover_df = layover_df.drop(columns=['time_point_order', 'point_type', 'standard_type'])
(layover_df.groupby(['block_id', 'service_date'])[['theoretical_layover', 'actual_layover']]
            .agg({'theoretical_layover': ['mean', 'max', 'count'],
                  'actual_layover': ['mean', 'max', 'count'],
                  'stop_id': 'first'})
)

In [ ]:
test = adt_route10.groupby(['half_trip_id'])
startpoint_df = test.get_group